<a href="https://colab.research.google.com/github/i-am-manish/BioBERT--Ai-medical-assistant/blob/master/BioBERT_manish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![DoctorBert](https://snag.gy/oF84Gk.jpg)


![DoctorBert](https://i.ytimg.com/vi/nPemP-Q0Xn8/hqdefault.jpg)

This is a Colab Demo of our DocProduct Tensorflow 2.0 Hackathon Project

Project details can be seen on our Github repo
https://github.com/Santosh-Gupta/DocProduct

and our Devpost page
https://devpost.com/software/nlp-doctor

In order to fit the computational limits of Google Colab, we had to reduce the amount of searchable Q&A datapoints from ~700,000 to ~200,000.
We also converted our search embeddings from float32 to float16, although this does not seem to dimish the quality of search (*but that's a whole other product to explore*)

Instructions:

-Run the first cell to install TF2.0 and FAISS

-Run the second cell to download the models and data

-Run the third cell to load the weights and data

-Input your question and search parameters into the fourth cell to play around with our model results




In [3]:
#@title Install Faiss, TF 2.0, and our Github. Double Click to see code

#To use CPU FAISS use
!wget  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
#To use GPU FAISS use
# !wget  https://anaconda.org/pytorch/faiss-gpu/1.2.1/download/linux-64/faiss-gpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!tar xvjf faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf
!pip install https://github.com/re-search/DocProduct/archive/v0.2.0_dev.zip
!pip install gpt2-estimator
!pip install pyarrow


HTTP request sent, awaiting response... 200 OK
Length: 4106816 (3.9M) [application/x-tar]
Saving to: ‘faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2.3’

faiss-cpu-1.2.1-py3 100%[===================>]   3.92M  3.57MB/s    in 1.1s    

2019-09-14 10:10:54 (3.57 MB/s) - ‘faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2.3’ saved [4106816/4106816]

info/hash_input.json
info/has_prefix
info/index.json
info/git
info/files
info/LICENSE.txt
info/about.json
info/paths.json
lib/python3.6/site-packages/faiss-0.1-py3.6.egg-info/dependency_links.txt
lib/python3.6/site-packages/faiss-0.1-py3.6.egg-info/not-zip-safe
lib/python3.6/site-packages/faiss-0.1-py3.6.egg-info/requires.txt
lib/python3.6/site-packages/faiss-0.1-py3.6.egg-info/top_level.txt
lib/python3.6/site-packages/faiss-0.1-py3.6.egg-info/native_libs.txt
info/test/run_test.py
info/test/run_test.sh
info/test/tests/run_tests.sh
lib/python3.6/site-packages/faiss-0.1-py3.6.egg-info/SOURCES.txt
info/recipe/conda_build_config.yaml
info/recipe/build.sh
info/

In [0]:
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                
import os
import requests

import urllib.request

# Download the file from `url` and save it locally under `file_name`:
urllib.request.urlretrieve('https://github.com/naver/biobert-pretrained/releases/download/v1.0-pubmed-pmc/biobert_v1.0_pubmed_pmc.tar.gz', 'BioBert.tar.gz')

if not os.path.exists('BioBertFolder'):
    os.makedirs('BioBertFolder')
    
import tarfile
tar = tarfile.open("BioBert.tar.gz")
tar.extractall(path='BioBertFolder/')
tar.close()

file_id = '1uCXv6mQkFfpw5txGnVCsl93Db7t5Z2mp'

download_file_from_google_drive(file_id, 'Float16EmbeddingsExpanded5-27-19.pkl')

file_id = 'https://onedrive.live.com/download?cid=9DEDF3C1E2D7E77F&resid=9DEDF3C1E2D7E77F%2132792&authkey=AEQ8GtkcDbe3K98'
    
urllib.request.urlretrieve( file_id, 'DataAndCheckpoint.zip')

if not os.path.exists('newFolder'):
    os.makedirs('newFolder')

import zipfile
zip_ref = zipfile.ZipFile('DataAndCheckpoint.zip', 'r')
zip_ref.extractall('newFolder')
zip_ref.close()

In [5]:
from docproduct.predictor import RetreiveQADoc

pretrained_path = 'BioBertFolder/biobert_v1.0_pubmed_pmc/'
# ffn_weight_file = None
bert_ffn_weight_file = 'newFolder/models/bertffn_crossentropy/bertffn'
embedding_file = 'Float16EmbeddingsExpanded5-27-19.pkl'

doc = RetreiveQADoc(pretrained_path=pretrained_path,
ffn_weight_file=None,
bert_ffn_weight_file=bert_ffn_weight_file,
embedding_file=embedding_file)

Failed to load GPU Faiss: No module named 'faiss.swigfaiss_gpu'
Faiss falling back to CPU-only.


In [8]:
#@title Type in your question (512 word limit) and search search parameters

question_text = "i am alone and i dont believe in anyone"  #@param {type:"string"}

search_similarity_by = 'answer'  #@param ['answer', "question"]

number_results_toReturn=10 #@param {type:"number"}

answer_only=True #@param ["False", "True"] {type:"raw"}

returned_results = doc.predict( question_text ,
                  search_by=search_similarity_by, topk=number_results_toReturn, answer_only=answer_only)

print('')
for jk in range(len(returned_results)):
    print("Result ", jk+1)
    print(returned_results[jk])
    print('')


1it [00:01,  1.37s/it]


Result  1
Have you found a psychologist/psychiatrist for these irrational fears yet?

Result  2
get yourself some help dealing with anxiety

Result  3
Isn't there some kind of charity you can give this stuff to? then they can bring the stuff to third world countries

Result  4
get a new group of friends

Result  5
Sounds like it's time to see someone. Taking medications to "knock you out" isn't going to solve your anxiety about the situation. Do you already have a therapist that you've seen in the past?,I'm sorry you've had such bad experiences with therapists. Have you tried something like meditation? It might help you learn to relax your body to help you sleep.

Result  6
there is no evidence to support.

Result  7
Have you thought about seeing someone about your anxiety? Sounds like you could really benefit from therapy and/or medication to help you deal with these fears.,I think it might really help. Being in a "constant state of anxiety" isn't great for your health, both physical